# WebCrawler Test
WebCrawler는 기본적으로 http 통신을 통해서 얻을 수 있는 HTML 데이터에서 우리가 원하는 데이터를 뽑아내는 과정이라고 볼 수 있다. <br>
python에서는 request 라는 통신 라이브러리와 BeauifulSoup라는 Common한 수준에서의 HTML Tag를 추출하는 기능으르 제공하는 라이브러리를 사용하여 간단하게 구현해 볼 수 있다. 기본적으로 추출하고자 하는 데이터가 Json 이나 XML 처럼 정규화된 형태가 없기 때문에 각 사이트 별로 로직을 다르게 구성할 필요가 있을 것이라고 생각된다. 

In [7]:
import requests
from bs4 import BeautifulSoup
import re, json, os, random
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce 
import pandas as pd
from io import StringIO

# url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
# nn_id = "nn123"
# nn_wf_ver_id ="1"

print("import common done")

import common done


# 정규 표현식 테스트 (Preprocessing 적용)
정규표현식은 Crawler 에서 필수적인 기능으로 사용법 몇 가지를 테스트해보도록 하겠다. 아래와 같이 몇가지 정규 표현식 예제들을 만들어 보았다. 정규 표현식은 매우 유용하지만 Cralwer 를 개발시에는 조금 불편한 부분들이 있을 수 있다. 이러한 경우 BeautifulSoup 라는 라이브러리를 사용하여 간단하게 사용할 수도 있지만, 정규표현식은 여전히 중요하다고 생각된다. 
- ^ : 시작 
- $ : 끝
- [] : 문자 , 예) [a-z]
- {최소, 최대} : 예) [a-z]{2,3} 
- '+' : '{1,}'
- '?' : '{0,1}'
- '*' : '{0,}'  
- '.' : 모든 문자 가능
- \d : 모든 숫자 
- \w : 모든 문자 
- [^a] : a를 제외한 ! 

In [8]:
import re 

data = "aaabbbcccddsgjs adjkfeklsjdfk jvsklfjsdklf jsdfadffsdf"
tel_no_list = "010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333"
virtual_html = "<html><body><title>abcd</title><p>가나다라</p><p>마바사아</p></body></html>"

#알파뱃(a~z) 중에서 단어를 찾는다 
reg = re.compile('[a-z]*')
print("1.알파뱃 찾기")
print("IN : {0}".format(reg.findall(data)))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

#알파뱃(a~z) 중에서 두단어를 찾는다 
reg = re.compile('[a-z]{0,5}')
print("2.알파뱃 두단어 찾기")
print("IN : {0}".format(reg.findall(data)))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

# 특정 패턴에 해당하는 전화번호를 다 찾아보자 
reg = re.compile('\d{2,3}-\d{3,4}-\d{4,4}')
print("3.전화번호 패턴 추출")
print("IN : {0}".format(tel_no_list))
print("OUT : {0}".format(reg.findall(tel_no_list)))
print('\n')

# 특정 패턴에 해당하는 전화번호를 다 찾아보자 
# 3333-3333 도 찾아보자 
reg = re.compile('(\d{2,3}-\d{3,4}-\d{4,4}|\d{3,4}-\d{4,4})')
out = reg.findall(tel_no_list)
print("4.전화번호 패턴 추출 (or 문 사용)")
print("IN : {0}".format(tel_no_list))
print("OUT : {0}".format(reg.findall(tel_no_list)))
print('\n')

# 특정 문자를 제외하고 
reg = re.compile('[^a]{1,1}[\w]+$')
print("5.특정 패턴 제외하고 찾기")
print("IN : {0}".format(data))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

# 한글 전체 찾기
reg = re.compile('[가-힣]{1,}')
print("5.한글만 다 찾아 보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

# title 태그 안에 있는 것만 다 가지고 와보자
# (xxx) 은 추출이다 
reg = re.compile('<title[^>]*>([^<]+)</title>')
print("6.Title Tag만 다 찾아보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

# p 태그 안에 있는 것만 다 가지고 와보자
reg = re.compile('<p>([\w]+)</p>')
print("7.P Tag만 다 찾아보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

1.알파뱃 찾기
IN : ['aaabbbcccddsgjs', '', 'adjkfeklsjdfk', '', 'jvsklfjsdklf', '', 'jsdfadffsdf', '']
OUT : ['aaabbbcccddsgjs', '', 'adjkfeklsjdfk', '', 'jvsklfjsdklf', '', 'jsdfadffsdf', '']


2.알파뱃 두단어 찾기
IN : ['aaabb', 'bcccd', 'dsgjs', '', 'adjkf', 'eklsj', 'dfk', '', 'jvskl', 'fjsdk', 'lf', '', 'jsdfa', 'dffsd', 'f', '']
OUT : ['aaabb', 'bcccd', 'dsgjs', '', 'adjkf', 'eklsj', 'dfk', '', 'jvskl', 'fjsdk', 'lf', '', 'jsdfa', 'dffsd', 'f', '']


3.전화번호 패턴 추출
IN : 010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333
OUT : ['010-9999-9999', '019-2222-4444', '082-1111-3333', '02-111-1111']


4.전화번호 패턴 추출 (or 문 사용)
IN : 010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333
OUT : ['010-9999-9999', '019-2222-4444', '082-1111-3333', '02-111-1111', '3333-3333']


5.특정 패턴 제외하고 찾기
IN : aaabbbcccddsgjs adjkfeklsjdfk jvsklfjsdklf jsdfadffsdf
OUT : [' jsdfadffsdf']


5.한글만 다 찾아 보기
IN : <html><body><title>abcd</title><p>가나다라</p><p>마바사아</p></body></html>
OUT : ['가

# BeautifulSoup 간단하게 연결해 보기
아래는 간단하게 Naver.com 에 접근하여 Title Tag 값을 추출하는 예제이다. 보는 것처럼 requests 를 호출해서 HTML 데이터를 얻은 후 이 데이터를
BeautifulSoup 를 사용해서 필요한 Tag 를 손쉽게 추출 할 수 있다. 정규식아라고 하면 아래와 같은 형태로 구현이 되어야 할 것이지만 라이브러리를 사용하면 더욱 손 쉽게 사용가능하다. <br>
- reg = re.compile("<title[^>]*>([^<]+)</title>") <br>
- soup.find_all('title')<br>

In [9]:
def crawler(iter) : 
    url = "http://naver.com"
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text , 'lxml')
    for raw in soup.find_all('title') : 
        print(raw)
crawler(1)

<title>NAVER</title>


# BeautifulSoup 간단하게 Table Parsing 해보기
HTML Table 을 Parsing 해서 간단하게 csv 로 바꿔보자. BeautifulSoup 이 얼마나 편한지 알 수 있다. 마치 Jquery 를 하는것 처럼 Elements 를 따라가면서 Find method 를 사용하고 마치 JQuery Iter 작업 처럼 복수의 요소를 쉽게 검색할 수 있다. 

In [20]:
# 엄청간단하게 table 을 파싱할 수 있다 
def crawler_csv() : 
    url = "https://race.kra.co.kr/raceScore/ObjtRaceRaceList.do?Act=04&Sub=3&meet=3"
    return_line = []
    return_header = ""
    return_td = ""
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    
    div = soup.find('div', class_="tableType2")
    for th in div.find_all('th') : 
        return_header = return_header + th.text + ','
    for tr in div.find_all('tr') :
        for td in tr.find_all('td') : 
            return_td = return_td + td.text + ','
        return_td = return_td.rstrip(',') + '\\n'
    return (return_header.rstrip(',') + '\\n' + return_td)
    
test_set = crawler_csv()
# pandas 를 이용해서 컨버팅한 데이터가 정상인지 한번 확인해 보자
test_data = StringIO(test_set)
df = pd.read_csv(test_data, sep=",")
df.to_csv("./table_1.csv", sep=',', encoding='utf-8')
print("csv save done")

csv save done


# Site 에 존재하는 복수의 테이블을 저장해 보자
물론 여기서는 Table , Tr, Td 의 전형적인 구조라고 가정한다. 

In [19]:
def save_as_csv(data) :
    """
    랜덤하게 csv 이름을 생성하여 주어진 데이터를 저장 
    """
    rand_name = random.randrange(1,10000)
    save_data = StringIO(data)
    df = pd.read_csv(save_data, sep=",")
    df.to_csv("./" + str(rand_name) + ".csv", sep=',', encoding='utf-8')
    
def table_to_csv(url) : 
    """
    전형적인 형태의 Table 을 Parsing 하여 csv 로 저장한다 
    """
    return_line = []
    return_td = ""
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    for table in soup.find_all('table') :
        for tr in table.find_all('tr') :
            for td in table.find_all('td') :
                return_td = return_td + td.text + ','
            return_td = return_td.rstrip(',') + '\\n'
        # save each table 
        save_as_csv(return_td)
        return_td = ""
    
table_to_csv("https://ko.wikipedia.org/wiki/")
print("table csv save done")

# Wikipedia 한글 페이지 Link를 따라가면서 데이터 수집
아주 간단한 WebCralwer 를 테스트 해보았다. 간단하게 WikiPedia 한글 사이트를 처음으로 시작해서 해당 사이트에 존재하는 "P" 태그를 수집하고 해당 페이지에서 존재하는 Link를 찾아서 이동하고, 해당 페이지에서 "P" 태그를 찾아서 저장하는 행위를 반복하는 코드이다.  <br>
별도의 정규 표현식을 이용하고 싶을 경우에는 정규 표현식을 파라메터로 받아서 해당 정규 표현식으로 Crawler 작업을 실행한다. <br> 
메서드는 spider(2, 'https://ko.wikipedia.org/wiki/') 형태로 되어 있으며, 첫 번째 파레메터는 Inception Level로 몇번이나, Link를 따라 들어가서 Crawler 작업을 수행할 것인지를 지정하는 작업이고, 두 번째 파라메터는 시작할 사이트의 주소가 되겠다. 실제로 실행시 지정한 페이지뿐만 아니라 연결된 Link들을 계속 찾아서 필요한 데이터를 추출하는 것을 볼 수 있다. 세번째 파라메터로 정규 표현식을 입력 받으며, 해당 값이 있는 경우 정규 표현식을 활용하여 파싱 작업을 수행한다. 

In [22]:
def task(page, max_pages, url_path, file_w, reg = None):
    """
    지정된 수만큼 제귀 형태로 모든 링크를 따라가서 전부 수집한다. 
    """
    if page == max_pages :
        get_single_article(url_path, file_w, reg_exp=str(reg))
        table_to_csv(url_path)
    else : 
        get_single_article(url_path, file_w, reg_exp=str(reg))
        table_to_csv(url_path)
        source_code = requests.get(url_path)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        page += 1
        for link in soup.find_all('a'):
            href = link.get('href')
            if (href != None and re.search("https://ko", href)) : 
                task(page, max_pages, href, file_w, reg=str(reg))

def get_single_article(item_url, file_w, reg_exp = None):
    """
    p 태그를 가지고와서 파싱하거나 
    지정된 reg_exp 를 사용하여 파싱한다 
    """
    print("href : {0}".format(item_url))
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    
    if(reg_exp) : 
        #정규 표현식이 있는 경우 해당 정규 표현식에 맞는 데이터를 추출 
        reg = re.compile(reg_exp)
        for contents in reg.findall(plain_text):
            file_w.write(contents)
    else : 
        #별도의 Regex가 없는 경우 p tag 에 있는 모든 데이터 추출 
        for contents in soup.find_all('p'):
            file_w.write(contents.text)

def spider(max_pages, url_path, path = "/home/dev/wiki/", file_name='test.txt', reg_exp = None) :
    """
    본 Function 을 실행하면 WikiPedia 첫 페이지에서 실행해서 
    지정된 횟수만큼 페이지를 따라 들어가서 정해진 패턴을 수집한다. 
    max_pages : 몇번 Page를 따라 들어갈 것인가를 정의하는 변수 
    """
    if not os.path.exists(path):
        os.makedirs(path)
    with open(''.join([path, file_name]), "w") as file_w :   
        print("# Job Start!!")
        task(1, max_pages, url_path, file_w, reg = reg_exp)
        print("# Job Done!!")

def save_as_csv(data) :
    """
    랜덤하게 csv 이름을 생성하여 주어진 데이터를 저장 
    """
    rand_name = random.randrange(1,10000)
    save_data = StringIO(data)
    df = pd.read_csv(save_data, sep=",")
    df.to_csv("./" + str(rand_name) + ".csv", sep=',', encoding='utf-8')
    print("file saved as : {0}".format(str(rand_name)))
    
def table_to_csv(url) : 
    """
    전형적인 형태의 Table 을 Parsing 하여 csv 로 저장한다 
    """
    try : 
        return_line = []
        return_td = ""
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        for table in soup.find_all('table') :
            for tr in table.find_all('tr') :
                for td in table.find_all('td') :
                    return_td = return_td + td.text + ','
                return_td = return_td.rstrip(',') + '\\n'
            # save each table 
            save_as_csv(return_td)
            return_td = ""
    except Exception as e : 
        return True
        
# 주어진 횟수만큼 해당 사이트를 시작으로 크롤링 시작 
# first parm : Inception 횟수 
# second parm : initial site 
# reg_exp : 정규 표현식 사용 가능 

# (1) 정규 표현식 사용 CASE (한글 전체 추출)
spider(2, 'https://ko.wikipedia.org/wiki/', reg_exp ='[가-힣\s]{1,}')
#spider(2, 'https://ko.wikipedia.org/wiki/', reg_exp ='<title[^>]*>([^<]+)</title>')

# (2) 정규 표현식 사용하지 않고 P 태크 추출
#spider(2, 'https://ko.wikipedia.org/wiki/')

# Job Start!!
href : https://ko.wikipedia.org/wiki/
file saved as : 3351
file saved as : 8723
file saved as : 1269
file saved as : 5878
href : https://ko.wiktionary.org/wiki/
href : https://ko.wiktionary.org/wiki/
href : https://ko.wikinews.org/wiki/
href : https://ko.wikinews.org/wiki/
href : https://ko.wikisource.org/wiki/
href : https://ko.wikisource.org/wiki/
href : https://ko.wikiversity.org/wiki/
file saved as : 9810
file saved as : 9706
file saved as : 3516
href : https://ko.wikiversity.org/wiki/
file saved as : 4979
file saved as : 4298
file saved as : 4276
href : https://ko.wikivoyage.org/wiki/
file saved as : 2667
href : https://ko.wikivoyage.org/wiki/%EB%8C%80%EB%AC%B8
file saved as : 7796
href : https://ko.wikiquote.org/wiki/
file saved as : 8306
file saved as : 7449
file saved as : 2191
file saved as : 9446
file saved as : 6062
href : https://ko.wikiquote.org/wiki/
file saved as : 1220
file saved as : 9043
file saved as : 5517
file saved as : 4447
file saved as : 6692
href